In [1]:
# Import dependencies.
import opendatasets as od
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine
import psycopg2
import io
import config as creds

In [2]:
# Download dataset from Kaggle. 
# Kaggle username and API Key required.
od.download("https://www.kaggle.com/datasets/mrisdal/fake-news")

Skipping, found downloaded files in ".\fake-news" (use force=True to force download)


In [3]:
# Download second dataset from Kaggle.
od.download("https://www.kaggle.com/datasets/ruchi798/source-based-news-classification")

Skipping, found downloaded files in ".\source-based-news-classification" (use force=True to force download)


In [4]:
# Load first dataset into a Pandas dataframe.
file =('fake-news/fake.csv')
fake_df = pd.read_csv(file)
fake_df

,uuid,ord_in_thread,author,published,title,text,language,crawled,site_url,country,domain_rank,thread_title,spam_score,main_img_url,replies_count,participants_count,likes,comments,shares,type
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,Muslims BUSTED: They Stole Millions In Gov’t B...,Print They should pay all the back all the mon...,english,2016-10-27T01:49:27.168+03:00,100percentfedup.com,US,25689.0,Muslims BUSTED: They Stole Millions In Gov’t B...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,reasoning with facts,2016-10-29T08:47:11.259+03:00,Re: Why Did Attorney General Loretta Lynch Ple...,Why Did Attorney General Loretta Lynch Plead T...,english,2016-10-29T08:47:11.259+03:00,100percentfedup.com,US,25689.0,Re: Why Did Attorney General Loretta Lynch Ple...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,BREAKING: Weiner Cooperating With FBI On Hilla...,Red State : \nFox News Sunday reported this mo...,english,2016-10-31T01:41:49.479+02:00,100percentfedup.com,US,25689.0,BREAKING: Weiner Cooperating With FBI On Hilla...,0.000,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,0,1,0,0,0,bias
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,Fed Up,2016-11-01T05:22:00.000+02:00,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,Email Kayla Mueller was a prisoner and torture...,english,2016-11-01T15:46:26.304+02:00,100percentfedup.com,US,25689.0,PIN DROP SPEECH BY FATHER OF DAUGHTER Kidnappe...,0.068,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,Fed Up,2016-11-01T21:56:00.000+02:00,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,Email HEALTHCARE REFORM TO MAKE AMERICA GREAT ...,english,2016-11-01T23:59:42.266+02:00,100percentfedup.com,US,25689.0,FANTASTIC! TRUMP'S 7 POINT PLAN To Reform Heal...,0.865,http://100percentfedup.com/wp-content/uploads/...,0,0,0,0,0,bias
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12994,f1b5d0e44803f48732bde854a9fdf95837219b12,2,replaceme,2016-10-26T23:58:00.000+03:00,NaN,It DOES allow you to put a dog face on top of ...,english,2016-10-27T00:37:46.194+03:00,zerohedge.com,US,2435.0,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs
12995,36011ceba3647e1bea78299b68b6fb705a1fc1ad,3,Freedumb,2016-10-27T00:02:00.000+03:00,NaN,Wait till you see what happens to the valuatio...,english,2016-10-27T00:37:46.220+03:00,zerohedge.com,US,2435.0,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs
12996,6995d1aa9ac99926106489b14b5530e85358059a,4,major major maj...,2016-10-27T00:06:00.000+03:00,NaN,I'm waiting for the one that puts a pussy on m...,english,2016-10-27T00:37:46.244+03:00,zerohedge.com,US,2435.0,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs
12997,7de8ae90eee164eb756db6c8a3772288e11d7a94,5,beemasters,2016-10-27T00:09:00.000+03:00,NaN,$4 Billion even after they are known to be kee...,english,2016-10-27T00:37:46.247+03:00,zerohedge.com,US,2435.0,"Snapchat To Raise Up To $4 Billion In IPO, Val...",0.000,NaN,40,32,0,0,0,bs


In [5]:
# Load second dataset into a Pandas dataframe.
file2 = ('source-based-news-classification/news_articles.csv')
news_articles_df = pd.read_csv(file2)
news_articles_df

,author,published,title,text,language,site_url,main_img_url,type,label,title_without_stopwords,text_without_stopwords,hasImage
0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,muslims busted stole millions govt benefits,print pay back money plus interest entire fami...,1.0
1,reasoning with facts,2016-10-29T08:47:11.259+03:00,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...,1.0
2,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,breaking weiner cooperating fbi hillary email ...,red state fox news sunday reported morning ant...,1.0
3,Fed Up,2016-11-01T05:22:00.000+02:00,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,pin drop speech father daughter kidnapped kill...,email kayla mueller prisoner tortured isis cha...,1.0
4,Fed Up,2016-11-01T21:56:00.000+02:00,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,fantastic trumps point plan reform healthcare ...,email healthcare reform make america great sin...,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2091,-NO AUTHOR-,2016-10-27T15:36:10.573+03:00,teens walk free after gangrape conviction,NaN,english,wnd.com,http://www.wnd.com/files/2016/10/hillary_haunt...,bias,Real,good samaritan wearing indian headdress disarm...,NaN,1.0
2092,-NO AUTHOR-,2016-10-27T15:36:10.671+03:00,school named for munichmassacre mastermind,NaN,english,wnd.com,http://www.wnd.com/files/2016/10/rambo_richard...,bias,Real,skype sex scam fortune built shame,NaN,1.0
2093,-NO AUTHOR-,2016-10-27T13:30:00.000+03:00,russia unveils satan missile,NaN,english,wnd.com,http://www.wnd.com/files/2016/10/skype_sex_sca...,bs,Fake,cannabis aficionados develop thca crystalline ...,NaN,1.0
2094,-NO AUTHOR-,2016-10-27T15:58:41.935+03:00,check out hillarythemed haunted house,NaN,english,wnd.com,http://worldtruth.tv/wp-content/uploads/2016/1...,bs,Fake,title,NaN,0.0


In [6]:
# Examine the data in the first dataframe before cleaning.
fake_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12999 entries, 0 to 12998
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   uuid                12999 non-null  object 
 1   ord_in_thread       12999 non-null  int64  
 2   author              10575 non-null  object 
 3   published           12999 non-null  object 
 4   title               12319 non-null  object 
 5   text                12953 non-null  object 
 6   language            12999 non-null  object 
 7   crawled             12999 non-null  object 
 8   site_url            12999 non-null  object 
 9   country             12823 non-null  object 
 10  domain_rank         8776 non-null   float64
 11  thread_title        12987 non-null  object 
 12  spam_score          12999 non-null  float64
 13  main_img_url        9356 non-null   object 
 14  replies_count       12999 non-null  int64  
 15  participants_count  12999 non-null  int64  
 16  like

In [7]:
# Examine the data in the second dataframe before cleaning.
news_articles_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2096 entries, 0 to 2095
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   author                   2096 non-null   object 
 1   published                2096 non-null   object 
 2   title                    2096 non-null   object 
 3   text                     2050 non-null   object 
 4   language                 2095 non-null   object 
 5   site_url                 2095 non-null   object 
 6   main_img_url             2095 non-null   object 
 7   type                     2095 non-null   object 
 8   label                    2095 non-null   object 
 9   title_without_stopwords  2094 non-null   object 
 10  text_without_stopwords   2046 non-null   object 
 11  hasImage                 2095 non-null   float64
dtypes: float64(1), object(11)
memory usage: 196.6+ KB


In [8]:
# Begin cleaning first dataset.
# Drop 'domain_rank' column to avoid removing too many null rows
fake_df_clean = fake_df.drop('domain_rank', axis=1)

In [9]:
# Drop null values from first dataset.
fake_df_clean = fake_df_clean.dropna()

In [10]:
# Keep only rows with titled articles (drop rows where article title is "no title")
fake_df_clean = fake_df_clean[fake_df_clean.title != "no title"]

In [11]:
# Alter title column so that titles from fake_df_clean match titles from news_articles_df
# Remove uppercase letters
fake_df_clean['title'] = fake_df_clean['title'].str.lower()

In [12]:
# Remove punctuation
fake_df_clean['title'] = fake_df_clean['title'].str.replace('[^\w\s]','')

C:\Users\human\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [13]:
# Remove any numerical digits
fake_df_clean['title'] = fake_df_clean['title'].str.replace('\d+', '')

C:\Users\human\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [14]:
# Remove any rows with duplicate titles from the first dataset
fake_df_clean.drop_duplicates(subset="title", keep = 'first', inplace = True)

In [15]:
# Examine cleaned dataset
fake_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6208 entries, 0 to 12898
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   uuid                6208 non-null   object 
 1   ord_in_thread       6208 non-null   int64  
 2   author              6208 non-null   object 
 3   published           6208 non-null   object 
 4   title               6208 non-null   object 
 5   text                6208 non-null   object 
 6   language            6208 non-null   object 
 7   crawled             6208 non-null   object 
 8   site_url            6208 non-null   object 
 9   country             6208 non-null   object 
 10  thread_title        6208 non-null   object 
 11  spam_score          6208 non-null   float64
 12  main_img_url        6208 non-null   object 
 13  replies_count       6208 non-null   int64  
 14  participants_count  6208 non-null   int64  
 15  likes               6208 non-null   int64  
 16  comme

In [16]:
# Begin cleaning second dataset
# Remove rows with null values
news_articles_df_clean = news_articles_df.dropna()

In [17]:
# Keep only rows with titled articles (drop rows where article title is "no title")
news_articles_df_clean = news_articles_df_clean[news_articles_df_clean.title != "no title"]

In [18]:
# Remove any rows with duplicated titles in the second dataset
news_articles_df_clean.drop_duplicates(subset="title", keep = 'first', inplace = True)

In [19]:
# Examine cleaned dataset
news_articles_df_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1754 entries, 0 to 2037
Data columns (total 12 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   author                   1754 non-null   object 
 1   published                1754 non-null   object 
 2   title                    1754 non-null   object 
 3   text                     1754 non-null   object 
 4   language                 1754 non-null   object 
 5   site_url                 1754 non-null   object 
 6   main_img_url             1754 non-null   object 
 7   type                     1754 non-null   object 
 8   label                    1754 non-null   object 
 9   title_without_stopwords  1754 non-null   object 
 10  text_without_stopwords   1754 non-null   object 
 11  hasImage                 1754 non-null   float64
dtypes: float64(1), object(11)
memory usage: 178.1+ KB


In [20]:
# Merge cleaned datasets to 
merged_df = pd.merge(news_articles_df_clean, fake_df_clean[['title', 'uuid', 'spam_score', 'country',
                                                                        'replies_count', 'participants_count',
                                                                        'likes', 'comments', 'shares', 'ord_in_thread']], on='title', how='inner')
merged_df

,author,published,title,text,language,site_url,main_img_url,type,label,title_without_stopwords,...,hasImage,uuid,spam_score,country,replies_count,participants_count,likes,comments,shares,ord_in_thread
0,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,muslims busted stole millions govt benefits,...,1.0,6a175f46bcd24d39b3e962ad0f29936721db70db,0.000,US,0,1,0,0,0,0
1,reasoning with facts,2016-10-29T08:47:11.259+03:00,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,attorney general loretta lynch plead fifth,...,1.0,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0.000,US,0,1,0,0,0,0
2,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,bias,Real,breaking weiner cooperating fbi hillary email ...,...,1.0,c70e149fdd53de5e61c29281100b9de0ed268bc3,0.000,US,0,1,0,0,0,0
3,Fed Up,2016-11-01T05:22:00.000+02:00,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,pin drop speech father daughter kidnapped kill...,...,1.0,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0.068,US,0,0,0,0,0,0
4,Fed Up,2016-11-01T21:56:00.000+02:00,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,bias,Real,fantastic trumps point plan reform healthcare ...,...,1.0,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0.865,US,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,-NO AUTHOR-,2016-10-27T01:02:34.523+03:00,federal judge charges sheriff joe with contempt,rand paul returns to us treasury i promised k...,english,wnd.com,http://www.wnd.com/files/2014/02/miriam-carey-...,bias,Real,representative government thugocracy,...,0.0,dd356f03a743b795c3ebe8562160cf3347f677d6,0.000,GB,0,1,0,0,0,0
972,-NO AUTHOR-,2016-10-27T00:52:00.000+03:00,all about jesus shirt sparks school uproar,print \nwhen judith gait met father x it was a...,english,wnd.com,No Image URL,bias,Real,never trumpers must reconsider,...,0.0,37903d3104f269c41250752b30d469189f86222e,0.000,GB,0,1,0,0,0,0
973,Joseph Farah,2016-10-27T02:38:20.630+03:00,loretta lynch takes on killer cops except in dc,donations world wildlife falls by in years c...,english,wnd.com,No Image URL,bias,Real,election crossroads socialism capitalism,...,0.0,a631576d9ebcdc01c623a7cbdd1c370245b17433,0.000,GB,0,1,0,0,0,0
974,-NO AUTHOR-,2016-10-27T02:39:12.484+03:00,gingrich slutshames megyn kelly,obamacare tax penalty ill take it millions say...,english,wnd.com,http://mobile.wnd.com/files/2014/02/miriam-car...,bias,Real,representative government thugocracy,...,0.0,9eea614efdad6d5ef6a8c7f2ec4079aa60eecdfa,0.000,GB,0,1,0,0,0,0


In [21]:
# Examine merged dataset
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 976 entries, 0 to 975
Data columns (total 21 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   author                   976 non-null    object 
 1   published                976 non-null    object 
 2   title                    976 non-null    object 
 3   text                     976 non-null    object 
 4   language                 976 non-null    object 
 5   site_url                 976 non-null    object 
 6   main_img_url             976 non-null    object 
 7   type                     976 non-null    object 
 8   label                    976 non-null    object 
 9   title_without_stopwords  976 non-null    object 
 10  text_without_stopwords   976 non-null    object 
 11  hasImage                 976 non-null    float64
 12  uuid                     976 non-null    object 
 13  spam_score               976 non-null    float64
 14  country                  9

In [22]:
# Create a subset of the data to hold qualitative data about articles (strings and objects)
clean_articles_df = merged_df[['uuid', 'author', 'published', 'title', 'text', 'language', 'site_url',
                                           'main_img_url', 'country', 'type', 'label', 
                               'title_without_stopwords', 'text_without_stopwords']]
clean_articles_df

,uuid,author,published,title,text,language,site_url,main_img_url,country,type,label,title_without_stopwords,text_without_stopwords
0,6a175f46bcd24d39b3e962ad0f29936721db70db,Barracuda Brigade,2016-10-26T21:41:00.000+03:00,muslims busted they stole millions in govt ben...,print they should pay all the back all the mon...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,US,bias,Real,muslims busted stole millions govt benefits,print pay back money plus interest entire fami...
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,reasoning with facts,2016-10-29T08:47:11.259+03:00,re why did attorney general loretta lynch plea...,why did attorney general loretta lynch plead t...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,US,bias,Real,attorney general loretta lynch plead fifth,attorney general loretta lynch plead fifth bar...
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,Barracuda Brigade,2016-10-31T01:41:49.479+02:00,breaking weiner cooperating with fbi on hillar...,red state \nfox news sunday reported this mor...,english,100percentfedup.com,http://bb4sp.com/wp-content/uploads/2016/10/Fu...,US,bias,Real,breaking weiner cooperating fbi hillary email ...,red state fox news sunday reported morning ant...
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,Fed Up,2016-11-01T05:22:00.000+02:00,pin drop speech by father of daughter kidnappe...,email kayla mueller was a prisoner and torture...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,US,bias,Real,pin drop speech father daughter kidnapped kill...,email kayla mueller prisoner tortured isis cha...
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,Fed Up,2016-11-01T21:56:00.000+02:00,fantastic trumps point plan to reform healthc...,email healthcare reform to make america great ...,english,100percentfedup.com,http://100percentfedup.com/wp-content/uploads/...,US,bias,Real,fantastic trumps point plan reform healthcare ...,email healthcare reform make america great sin...
...,...,...,...,...,...,...,...,...,...,...,...,...,...
971,dd356f03a743b795c3ebe8562160cf3347f677d6,-NO AUTHOR-,2016-10-27T01:02:34.523+03:00,federal judge charges sheriff joe with contempt,rand paul returns to us treasury i promised k...,english,wnd.com,http://www.wnd.com/files/2014/02/miriam-carey-...,GB,bias,Real,representative government thugocracy,boycott targets trump family retail front part...
972,37903d3104f269c41250752b30d469189f86222e,-NO AUTHOR-,2016-10-27T00:52:00.000+03:00,all about jesus shirt sparks school uproar,print \nwhen judith gait met father x it was a...,english,wnd.com,No Image URL,GB,bias,Real,never trumpers must reconsider,yearold hebrew mention jerusalem found rare pa...
973,a631576d9ebcdc01c623a7cbdd1c370245b17433,Joseph Farah,2016-10-27T02:38:20.630+03:00,loretta lynch takes on killer cops except in dc,donations world wildlife falls by in years c...,english,wnd.com,No Image URL,GB,bias,Real,election crossroads socialism capitalism,donations world wildlife falls years critic th...
974,9eea614efdad6d5ef6a8c7f2ec4079aa60eecdfa,-NO AUTHOR-,2016-10-27T02:39:12.484+03:00,gingrich slutshames megyn kelly,obamacare tax penalty ill take it millions say...,english,wnd.com,http://mobile.wnd.com/files/2014/02/miriam-car...,GB,bias,Real,representative government thugocracy,pope god promised land people israel statement...


In [23]:
# Create a subset of the data to hold quantitative data about articles (integer and floats)
article_stats_df = merged_df[['uuid', 'ord_in_thread', 'hasImage', 'spam_score', 'replies_count', 'participants_count',
                                          'likes', 'comments', 'shares']]
article_stats_df

,uuid,ord_in_thread,hasImage,spam_score,replies_count,participants_count,likes,comments,shares
0,6a175f46bcd24d39b3e962ad0f29936721db70db,0,1.0,0.000,0,1,0,0,0
1,2bdc29d12605ef9cf3f09f9875040a7113be5d5b,0,1.0,0.000,0,1,0,0,0
2,c70e149fdd53de5e61c29281100b9de0ed268bc3,0,1.0,0.000,0,1,0,0,0
3,7cf7c15731ac2a116dd7f629bd57ea468ed70284,0,1.0,0.068,0,0,0,0,0
4,0206b54719c7e241ffe0ad4315b808290dbe6c0f,0,1.0,0.865,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...
971,dd356f03a743b795c3ebe8562160cf3347f677d6,0,0.0,0.000,0,1,0,0,0
972,37903d3104f269c41250752b30d469189f86222e,0,0.0,0.000,0,1,0,0,0
973,a631576d9ebcdc01c623a7cbdd1c370245b17433,0,0.0,0.000,0,1,0,0,0
974,9eea614efdad6d5ef6a8c7f2ec4079aa60eecdfa,0,0.0,0.000,0,1,0,0,0


In [24]:
# Create connection string to PostgreSQL database using credentials stored in a config file
conn_string = "postgresql://"+ creds.PGUSER +":"+ creds.PGPASSWORD +"@"+ creds.PGHOST +"/"+ creds.PGDATABASE

In [25]:
# Create an engine instance with sqlalchemy
db = create_engine(conn_string)
conn = db.connect()

In [26]:
# # Add dataframes to corresponding tables in PostgreSQL database
article_stats_df.to_sql('article_stats', con=conn, if_exists='replace', index=False)

In [27]:
clean_articles_df.to_sql('clean_articles', con=conn, if_exists='replace', index=False)

In [28]:
# Close connection
conn.close()